In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
import glob

In [26]:
import KMDHierarchicalClustering
from KMDHierarchicalClustering import cluster_scoring
from FlowCytometryTools import FCMeasurement
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, pairwise_distances
import pandas as pd

# Flow cytometry

In [362]:
data_name = 'Levine_32dim'
datafile = r'flow/raw_data/'+ data_name+ '.fcs'
sample = FCMeasurement(ID='Test Sample', datafile=datafile)
df = sample.data
df.shape

/Local/md_kaplan/anaconda3/envs/current/lib/python3.9/site-packages/fcsparser/api.py:440: UserWarning: The default channel names (defined by the $PnS parameter in the FCS file) were not unique. To avoid problems in downstream analysis, the channel names have been switched to the alternate channel names defined in the FCS file. To avoid seeing this warning message, explicitly instruct the FCS parser to use the alternate channel names by specifying the channel_naming parameter.
  warnings.warn(msg)


(265627, 41)

In [363]:
import pathlib
def get_y(seed):
    np.random.seed(seed)
    idx = []
    n = 20000
    i = 0
    while i < n:
        idx_new = np.random.randint(int(df.shape[0]), size=(1,))
        #if np.isnan(df['label'][idx_new].to_numpy()):
        #    continue
        #else:
        idx.append(int(idx_new[0]))
        i+=1
    idx = np.array(idx)
    y_true = np.array(df['label'][idx])

    y_pred_pattern = f"flow/subclust/{data_name}_y_pred_n_with_nan*20000_num_{seed}*.npy"
    print("Pattern:", y_pred_pattern)
    y_pred_filenames = glob.glob(y_pred_pattern)
    y_pred_filenames = sorted(y_pred_filenames, key = lambda x: -pathlib.Path(x).stat().st_mtime)[-1:]
    y_pred = [ np.load(x) for x in y_pred_filenames ]
    if y_pred[0].size > n:
        y_pred = [ x[idx] for x in y_pred ]
    
    return y_true, y_pred, df.iloc[idx]

In [364]:
score = []
count = []
for s in [1,2,3,4,5]:
    y_true, y_pred, _ = get_y(s)
    print(f"{s=} num of y_preds={len(y_pred)}")
    mask = np.isfinite(y_true)
    count.append(mask.sum())
    y_true = y_true[mask]
    score.append(max([ cluster_scoring.hungarian_acc(y_true, x[mask])[0] for x in y_pred ]))
print(score)
print(np.mean(score), np.mean(count)), count

Pattern: /storage/md_kaplan/azelig/subclust/Levine_32dim_y_pred_n_with_nan*20000_num_1*.npy
s=1 num of y_preds=1
Pattern: /storage/md_kaplan/azelig/subclust/Levine_32dim_y_pred_n_with_nan*20000_num_2*.npy
s=2 num of y_preds=1
Pattern: /storage/md_kaplan/azelig/subclust/Levine_32dim_y_pred_n_with_nan*20000_num_3*.npy
s=3 num of y_preds=1
Pattern: /storage/md_kaplan/azelig/subclust/Levine_32dim_y_pred_n_with_nan*20000_num_4*.npy
s=4 num of y_preds=1
Pattern: /storage/md_kaplan/azelig/subclust/Levine_32dim_y_pred_n_with_nan*20000_num_5*.npy
s=5 num of y_preds=1
[0.9334620334620335, 0.9312270016495369, 0.9334883120237634, 0.9048476276199049, 0.9357740847182306]
0.9277598118946939 7834.8


(None, [7770, 7881, 7743, 7777, 8003])

In [365]:
s = 3 # Choose the seed that is closest to real results
y_true, y_pred, X = get_y(s)
print(f"{s=} num of y_preds={len(y_pred)}")
mask = np.isfinite(y_true)
# Remove technical columns like viability, file number, label, etc
#X = X.iloc[mask] # Levine_13dim
X = X.iloc[mask][X.columns[2:-5]] # Levine_32dim
#X = X.iloc[mask][X.columns[8:-7]] # Samusik_01
y_true = y_true[mask].astype(int)
y_pred = y_pred[0][mask]
dists = KMDHierarchicalClustering.KMDClustering.calc_dists({}, X, 'correlation')

Pattern: /storage/md_kaplan/azelig/subclust/Levine_32dim_y_pred_n_with_nan*20000_num_3*.npy
s=3 num of y_preds=1


In [ ]:
#plt.rcParams['figure.dpi'] = 300
np.random.seed(1)
cluster_scoring.tsne_presentation(dists, y_true, y_pred)

# Large dataset section

## Levine15_32

In [8]:
data_file = r"flow/raw_data/Levine_32dim.fcs"
df = FCMeasurement(ID='Test Sample', datafile=data_file).data

/Local/md_kaplan/anaconda3/envs/current/lib/python3.9/site-packages/fcsparser/api.py:440: UserWarning: The default channel names (defined by the $PnS parameter in the FCS file) were not unique. To avoid problems in downstream analysis, the channel names have been switched to the alternate channel names defined in the FCS file. To avoid seeing this warning message, explicitly instruct the FCS parser to use the alternate channel names by specifying the channel_naming parameter.
  warnings.warn(msg)


In [9]:
num = 2
np.random.seed(num)
idx = []
i = 0
while i < 5000:
    idx_new = np.random.randint(int(df.shape[0]), size=(1,))
    if np.isnan(df['label'][idx_new].to_numpy()):
        continue
    else:
        idx.append(int(idx_new[0]))
        i += 1
idx = np.array(idx)

In [12]:
X = df.to_numpy()[idx][:, 2:-5] # Levine_32dim
dists = KMDHierarchicalClustering.KMDClustering.calc_dists({}, X, 'correlation')
y_true_sub = df['label'][idx].astype(int).to_numpy()
y_pred_sub = np.load("flow/subclust/Levine_32dim_y_pred_n_5000_num_2.npy")

In [ ]:
#plt.rcParams['figure.dpi'] = 600
cluster_scoring.tsne_presentation(dists, y_true_sub, y_pred_sub)

## PBMC68K_Pure

In [65]:
data_file = "sampling/pbmc_pure/pure6_pca.csv"
df = pd.read_csv(data_file)
X = df.filter(like='PC').to_numpy()
y_true = df['label'].to_numpy()

In [66]:
pred_file = "sampling/pbmc_pure/pure6_kmd_seed1_sample5000_mcs50_n6.npz"
pred = np.load(pred_file)
y_pred = pred['y_pred']
idx = pred['idx_sub']

In [67]:
y_pred_sub = y_pred[idx]
y_true_sub = y_true[idx]
X_sub = X[idx]

In [68]:
_, y = cluster_scoring.hungarian_acc(y_true_sub, y_pred_sub)

In [71]:
dists = KMDHierarchicalClustering.KMDClustering.calc_dists({}, X_sub, 'correlation')

In [ ]:
#plt.rcParams['figure.dpi'] = 600
cluster_scoring.tsne_presentation(dists, y_true_sub, y_pred_sub)

## Simulated 1M

In [87]:
X = np.load('sampling/1Msim/1M_pca.npy')
y_true = np.load('sampling/1Msim/1M_y_true.npy', allow_pickle=True).astype(int)
y_pred = np.load("sampling/1Msim/1Msim/1M_y_pred.npy")
idx = np.load("sampling/1Msim/1Msim/1M_idx_sampled.npy")

In [88]:
y_pred_sub = y_pred[idx]
y_true_sub = y_true[idx]
X_sub = X[idx]

In [89]:
dists = KMDHierarchicalClustering.KMDClustering.calc_dists({}, X_sub, 'correlation')

In [ ]:
#plt.rcParams['figure.dpi'] = 600
cluster_scoring.tsne_presentation(dists, y_true_sub, y_pred_sub)